In [1]:
import os
import pandas as pd
from tqdm import tqdm

base_path = "/mnt/data/wangshu/hcarag/narrativeqa/data"
train_base = os.path.join(base_path, "train")
test_base = os.path.join(base_path, "test")

train_max = 1101
test_max = 354


def get_merged_res(path_suffix):
    all_df = pd.DataFrame()

    # Trian
    for i in tqdm(range(train_max + 1)):
        path = os.path.join(train_base, str(i), path_suffix)
        if os.path.exists(path):
            tmp_df = pd.read_csv(path)
            all_df = pd.concat([all_df, tmp_df])

    # Test
    for i in tqdm(range(test_max + 1)):
        path = os.path.join(test_base, str(i), path_suffix)
        if os.path.exists(path):
            tmp_df = pd.read_csv(path)
            all_df = pd.concat([all_df, tmp_df])
    
    metrics_attr = []
    for col in all_df.columns:
        if col.startswith("bleu"):
            metrics_attr.append(col)
        elif col.startswith("rouge"):
            metrics_attr.append(col)
        elif col.startswith("meteor"):
            metrics_attr.append(col)
        elif col.startswith("modify"):
            metrics_attr.append(col)
            
    sel_df = all_df[metrics_attr]
    avg_metrics = sel_df.mean()
    # print each metric
    for idx, val in avg_metrics.items():
        print(f"{idx}: {val*100:.4f}")
    return avg_metrics
    
# suffix = "qa_dataset/graphrag_local.csv"    
# suffix = "qa_dataset/villa_rag_llama3.1:8b4k_top5.csv"    
suffix = "qa_dataset/BM25_rag_llama3.1:8b4k_top5.csv"    
# suffix = "hcarag/hc_index_8b/qa/global_False_3_15_5_15_False_mr_QA_False_0.csv"    
avg_metrics = get_merged_res(suffix)

100%|██████████| 355/355 [00:01<00:00, 266.87it/s]

bleu_1pred: 2.0256
bleu_4pred: 0.0026
modify_bleu_4pred: 0.0052
bleu_1_smoothpred: 2.0256
bleu_4_smoothpred: 0.0342
modify_bleu_4_smoothpred: 0.1063
meteorpred: 4.8877
rouge_l_f1pred: 2.7946
rouge_l_precisionpred: 1.5887
rouge_l_recallpred: 29.6738


In [ ]:
base_path = (
    "/mnt/data/wangshu/hcarag/narrativeqa/data/train/0/qa_dataset/graphrag_global.csv"
)

tmp_df = pd.read_csv(base_path)
tmp_df.head(2)

In [2]:
import os
from tqdm import tqdm
# extract all used token from log file inference


def extract_all_token(base_dir, token_line_pattern: str, sel_file_pattern: str = None):
    all_tokens = 0
    all_files = os.listdir(base_dir)

    if sel_file_pattern is not None:
        all_files = [f for f in all_files if f.endswith(sel_file_pattern)]

    print(f"Total files: {len(all_files)}")
    
    for f in tqdm(all_files):
        f_path = os.path.join(base_dir, f)
        with open(f_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if token_line_pattern in line:
                    tokens = int(line.split()[-1])
                    all_tokens += tokens
                    break

    print(f"Total tokens: {all_tokens}")
    return all_tokens


# base_dir = "/home/wangshu/rag/hier_graph_rag/dataset/narrativeqa/eval/evaluate_t0.1_global_3_15_5_15_mr_QA_False_0"


# # GraphRAG
# base_dir = "/home/wangshu/rag/hier_graph_rag/other_baseline/graphrag/eval_narrative"
# # sel_file_pattern = "-docqa-toke_usage.log"
# sel_file_pattern = "-localdocqa-toke_usage.log"

# villa-rag
# base_dir = "/home/wangshu/rag/hier_graph_rag/other_baseline/villa-rag/narrative_eval_res"
# sel_file_pattern = "-5-llama3.1:8b4k.log"

base_dir = "/home/wangshu/rag/hier_graph_rag/other_baseline/simple_BM25/narrative_eval_res"
sel_file_pattern = "-5-llama3.1:8b4k.log"
token_line_pattern = "Total token"

extract_all_token(base_dir, token_line_pattern, sel_file_pattern=sel_file_pattern)

Total files: 1457


  0%|          | 0/1457 [00:00<?, ?it/s]

100%|██████████| 1457/1457 [00:00<00:00, 13792.29it/s]

Total tokens: 95242793


95242793

In [ ]:
train_base = ""

In [11]:
# extract index token from log file
import re
import pandas as pd

tmp_log_file_path = "/mnt/data/wangshu/hcarag/HotpotQA/graphrag_io/output/20240927-191252/reports/indexing-engine.log"


def extract_index_token(log_file_path: str):
    create_graph_line = "INFO dependencies for create_final_community_reports:"
    all_tokens = 0
    create_graph_tokens = 0
    get_create_graph = False
    token_pattern = re.compile(r'input_tokens=(\d+), output_tokens=(\d+)')
    
    with open(log_file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            match = token_pattern.search(line)
            if match:
                input_tokens = int(match.group(1))
                output_tokens = int(match.group(2))
                total_tokens = input_tokens + output_tokens
                all_tokens += total_tokens
            if (get_create_graph is False) and  (create_graph_line in line):
                get_create_graph = True
                create_graph_tokens = all_tokens
    print(f"Total tokens: {all_tokens}")
    print(f"Create graph tokens: {create_graph_tokens}")
    return all_tokens, create_graph_tokens

all_tokens, create_graph_tokens = extract_index_token(tmp_log_file_path)

Total tokens: 25536517
Create graph tokens: 14293472
